In [6]:
!pip install -U langchain_community tiktoken langchainhub scikit-learn langchain langgraph langchain-nomic langchain_openai python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [504]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing import Annotated, Literal, Sequence, TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import tools_condition
from langgraph.graph import END, StateGraph, START
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import END, StateGraph, START
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
import os
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langsmith import traceable
from langsmith.wrappers import wrap_openai

In [505]:
load_dotenv()

True

In [506]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USER")
NEO4J_PWD = os.getenv("NEO4J_PWD")

In [508]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PWD)
llm = ChatOpenAI(temperature=0.2, model="gpt-4")

In [529]:
simple_prompt = ChatPromptTemplate.from_template("Answer this question: {question}")
chain = simple_prompt | llm | StrOutputParser()
result = chain.invoke({"question": "Hi"})
print(result)

Hello! How can I assist you today?


In [530]:
graph.schema

'Node properties:\nRawImage {UUID: STRING, ground_truth: STRING, image_name: STRING}\nALERT {UUID: STRING, alert_name: STRING, timestamp: DATE_TIME, description: STRING, source_topic: STRING, event_data: STRING, priority: STRING}\nExperiment {start_time: INTEGER, experiment_id: STRING}\nEdgeDevice {device_id: STRING}\nUser {user_id: STRING}\nModel {model_id: STRING}\nRelationship properties:\nSUBMITTED_BY {submitted_time: INTEGER}\nEXECUTED_ON {submitted_time: INTEGER}\nPROCESSED_BY {image_decision: STRING, image_receiving_timestamp: DATE_TIME, image_store_delete_time: DATE_TIME, scores: STRING, image_scoring_timestamp: DATE_TIME, model_id: STRING, ingestion_timestamp: DATE_TIME, image_count: INTEGER}\nUSED_BY {start_time: INTEGER}\nThe relationships:\n(:RawImage)-[:PROCESSED_BY]->(:Experiment)\n(:Experiment)-[:SUBMITTED_BY]->(:User)\n(:Experiment)-[:EXECUTED_ON]->(:EdgeDevice)\n(:Experiment)-[:USED_BY]->(:Model)'

In [531]:
sysprompt = """You are an expert in writing Cypher queries for a Neo4j database. Write Cypher queries that avoid using directional edges. Instead of using arrows (-> or <-) for relationships, use undirected relationships by using double hyphens (--) and specify the relationship type within square brackets.
Make sure that all relationships in the queries are undirected, 
using double hyphens and square brackets to specify the relationship type.

For example, instead of:
MATCH (u:User {{user_id: 'jstubbs'}})-[:SUBMITTED_BY]->(e:Experiment)
RETURN COUNT(e) AS NumberOfExperimentsRunByJstubbs

You should write:
MATCH (u:User {{user_id: 'jstubbs'}})-[r:SUBMITTED_BY]-(e:Experiment)
RETURN COUNT(e) AS NumberOfExperimentsRunByJstubbs

"""
generation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sysprompt),
        ("human", "only return the cypher query for the question: \n {question} \n Schema: {schema}.",
        ),
    ]
)

cypher_generator = generation_prompt | llm | StrOutputParser()

In [532]:
question = "Give all experiments by jstubbs"
cypher_out = cypher_generator.invoke({"schema": graph.schema, "question": question})
print(cypher_out)

MATCH (u:User {user_id: 'jstubbs'})-[r:SUBMITTED_BY]-(e:Experiment)
RETURN e AS ExperimentsByJstubbs


In [511]:
cypher_generation_check_system = """You are a grader assessing whether a given cypherql query is syntactically correct. If it has directional edges in the query, it's not syntactically correct.
     Give a binary score 'yes' or 'no'. 'Yes' the query is syntactically correct """
# Data model
class CheckCypherGeneration(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Query is syntactically correct and grounded in the given schema, 'yes' or 'no'"
    )

syntax_grader = llm.with_structured_output(CheckCypherGeneration)

syntax_grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", cypher_generation_check_system),
        ("human", "Schema: \n\n {schema} \n\n LLM generation: {cypher_generation}"),
    ]
)

syntax_checker = syntax_grader_prompt | syntax_grader

In [495]:
syntax_test = syntax_checker.invoke({"schema": graph.schema, "cypher_generation": cypher_out})

In [512]:
top_k = 10
print(syntax_test)

binary_score='yes'


In [366]:
# Graph query tool
@tool
def get_data_from_kg(query: str):
    """Run the query on neo4j"""
    response = graph.query(query)[:top_k]
    return str(response).replace("{", "{{").replace("}", "}}")

tools = [get_data_from_kg]

In [367]:
db_response = get_data_from_kg(cypher_out)
print(db_response)

[{{'COUNT(e)': 1}}]


In [513]:
generation_test_system = """You are a grader assessing whether a given result from a graph database aligns with the question asked. 
     Give a binary score 'yes' or 'no'. 'Yes' the result answers the question  """
# Data model
class CheckAnswerGeneration(BaseModel):
    """Binary score for generated answer."""

    binary_score: str = Field(
        description="Result answers the question, 'yes' or 'no'"
    )

answer_grader_llm = llm.with_structured_output(CheckAnswerGeneration)

answer_grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generation_test_system),
        ("human", "Question: \n\n {question} \n\n Graph query response: {query_response}"),
    ]
)

answer_grader = answer_grader_prompt | answer_grader_llm

In [369]:
answer_grader_test = answer_grader.invoke({"question": question, "query_response": query_response})
print(answer_grader_test)

binary_score='no'


In [514]:
answer_generator_template = """You are tasked with generating a response to the question using the context information available in query that was run on a knowledge graph. Keep the output structured if possible.

question: 
{question}

graph query response: 
{db_response} 
 """

answer_generator_prompt = PromptTemplate(
    input_variables=["question", "db_response"], template=answer_generator_template
)

answer_generator = answer_generator_prompt | llm | StrOutputParser()

In [385]:
answer_generator_test = answer_generator.invoke({"question": question, "db_response": db_response})
print(answer_generator_test)

Response:
jstubbs ran 1 experiment.


In [515]:
class GraphState(TypedDict):
    """
    Represents the state of the graph
    """

    question: str
    generated_answer: str
    query_response: str
    cypher_generation: str

In [516]:
def generate_cypher(state):
    """
    Generate or regenerate the cypher query.
    """
    print("---GENERATING CYPHER QUERY---")
    user_question = state["question"]
    cypher_generation = state["cypher_generation"]
    
    if cypher_generation is not None:
        user_question = user_question + " Previously generated cypher was wrong which was: " + cypher_generation 
    
    generated_cypher = cypher_generator.invoke({"schema": graph.get_structured_schema, "question": user_question})
    print(state)
    return {"cypher_generation": generated_cypher, "question": user_question}


def decide_retrieve(state):
    """
    Test if the cypher query is correct
    :param state: 
    :return: 
    """
    cypher_generation = state["cypher_generation"]
    print("---DECIDING IF CYPHER QUERY IS SYNTACTICALLY CORRECT---")
    print(state)

    score = syntax_checker.invoke({"schema": graph.get_structured_schema, "cypher_generation": cypher_generation})
    grade = score.binary_score
    print(f'Grade: {grade}')
    if grade == "yes":
        #correct query generated
        print(
            "---DECISION: GENERATED CYPHER QUERY IS SYNTACTICALLY CORRECT---"
        )
        return "retrieve_data"
    else:
        # Not correct query, regenerate
        print("---DECISION: GENERATED CYPHER QUERY IS INCORRECT---")
        print(cypher_generation)
        
        return "generate_cypher"

def execute_query(state):
    print("---DECISION: EXECUTING QUERY ON GRAPH ---")
    print(state)
    query = state["cypher_generation"]
    response = graph.query(query)[:top_k]
    query_result = str(response).replace("{", "{{").replace("}", "}}")
    return {"query_response": query_result, "cypher_generation": query}

def generate_human_response(state):
    print("---GENERATING HUMAN LIKE RESPONSE ---")

    print(state)

    user_question = state["question"]
    query_response = state["query_response"]
    
    generated_answer = answer_generator.invoke({"question": user_question, "db_response": query_response})
    print("GENERATED:" + generated_answer)
    return {"query_response": query_response, "question": user_question, "generated_answer": generated_answer}

    

In [517]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate_cypher", generate_cypher)  # retrieve
workflow.add_node("execute_query", execute_query)  # grade documents
workflow.add_node("gen_human_response", generate_human_response)  # grade documents
# workflow.add_node("test_cypher_query", decide_retrieve)  # grade documents
# workflow.add_node("generate", generate)  # generatae
# workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "generate_cypher")
workflow.add_edge("generate_cypher", "execute_query")
workflow.add_edge("execute_query", "gen_human_response")
workflow.add_edge("gen_human_response", END)
# workflow.add_conditional_edges(
#     "generate_cypher",
#     decide_retrieve,
#     {
#         "retrieve_data": "execute_query",
#         "generate": "generate_cypher",
#     },
# )

# Compile
app = workflow.compile()

In [518]:
from pprint import pprint

# Run
inputs = {"question": "How many experiments were run by swithana?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generated_answer"])

---GENERATING CYPHER QUERY---
{'question': 'How many experiments were run by swithana?', 'generated_answer': None, 'query_response': None, 'cypher_generation': None}
"Node 'generate_cypher':"
'\n---\n'
---DECISION: EXECUTING QUERY ON GRAPH ---
{'question': 'How many experiments were run by swithana?', 'generated_answer': None, 'query_response': None, 'cypher_generation': 'The Cypher query for the question "How many experiments were run by swithana?" would be:\n\n```cypher\nMATCH (u:User {user_id: \'swithana\'})-[r:SUBMITTED_BY]-(e:Experiment)\nRETURN COUNT(e) AS NumberOfExperimentsRunBySwithana\n```'}


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'The': expected 'FOREACH', 'ALTER', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'MATCH', 'MERGE', 'NODETACH', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"The Cypher query for the question "How many experiments were run by swithana?" would be:"
 ^}